In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob

In [2]:
crew_dict = {"Scarify":["Scarify"], "Strip Site":["Strip Site"], 
             "Soil Correction":["Soil Correction", "OX","OE", "Over Ex", "Excavate & Re-compact", "Over-Ex", "Excavate & Recompact"],
             "Ex Retainining Wall":["Excavate Site Retaining", "Excavate MSE", "Exc Retaining", "Exc Site Retn Walls"],
             "Export":["Export", "Exp"], "Import":["Import", "Imp"],
             "Ex Frost footing": ["Exc Fro", "Excavate Fro", "Footing Excavation"], 
             "Ex Trash Enc":["Excavate Trash", "Exc Trash"],
             "Cut to Fill":["Cut To Fill", "Utility Spoils - Site Fill", "Building Pad Fill w/ Spoils"],
             "Cut to Stockpile":["Cut to Stockpile", "Load & Stockpile"],
             "Handle spoils":["Handle Utility Spoils", "Handle Foundation Spoils", "Handle Spoils", "Handle & Stockpile"],
             "Double Handle":["Double Handle"], "Cut in C&G":["Cut In C", "Cut-in C", "Grade C&G"],
             "Ex Elevator":["Excavate Elevator"],
             "Grade Landscape":["grade Landscape", "grade Green", "Landscape"],
             "Grade Parkinglots and Roads":["Grade Boulevards","Grade Asphalt", "Grade Parking", "Grade Pkg", "Grade Main Roads"],
             "Grade SW":["Grade Sidewalks", "Grade Walks", "Exterior Slabs", "grade Brick Pavers", "Grade SW"],
             "Export":["Export"],
             "Grade BLDG": ["Grade Bldg", "Grade Building", "Grade Structural"],
             "Drain Tile":["Drain"],
             "Backfill Int":["Backfill Int", "Footing Backfill"],
             "Backfill Ext":["Backfill Ext"],
             "Backfill Elevator":["Backfill Elevator"],
             "Backfill Trash":["Backfill Trash"], "Backfill Frost": ["Backfill Frost"],
             "High Backfill": ["High Backfill"], "Grade Class X":["Grade Class"],
             "Excavate Interior":["Excavate Int", "Excavate Thickened", "Excavate Grade Beams", "Exc Int"],
             "Excavate Exterior":["Excavate Ext", "Exc Ext"], "Grade Pond":["Basin Grading","Grade Pond" , "Grade Detention Pond"],
             "Assist shoring":["Assist Shoring"],
             "Stairs":["Stairs"], "Place Strippings":["Place Stripping"],
             "Street Sweep":["Street Sweeping"]}

In [3]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0], errors='coerce')
#     attendance = str(df.iloc[0,0]).split(' ',1)[1]
    return job, edit_date

def job_code(completed_path):
    files = []
    for item in os.listdir(completed_path):
        job_code = item.split(' ')[1]
        files.append(job_code)
    files= pd.unique(files).tolist()
    return files
    
    
def lat_old(files):
    lat_file_list = []
#     old_file_list = []
    for file in files:
        job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
        latest_file = max(job_files, key=os.path.getctime)
        # hold off on oldest files
#         oldest_file = min(job_files, key=os.path.getctime)
#         old_file_list.append((oldest_file))
        lat_file_list.append((latest_file))
#     old_file_list = pd.unique(old_file_list) 
    lat_file_list = pd.unique(lat_file_list) 
#, old_file_list
    return lat_file_list

def files_to_dfs(file_list):
    df_list = []
#     for item in file_list:
    for file in file_list:
        num_sheet = len(pd.ExcelFile(file).sheet_names)
        for sheet in range(0,num_sheet):
            xl = pd.read_excel(file ,sheet_name=sheet)
            df = pd.DataFrame(xl)
            job, date = job_date(df)
            df.columns = df.iloc[2]
            df = df.drop(df.index[:3])
            df = df.iloc[:, :27]
            df['updated'] = date
            df['job'] = job
            df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df = df.dropna(subset=['Qty'])
            df = df.reset_index(drop=True)
            df_list.append(df)
    full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
    return full_df

In [4]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
c_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
ew = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\Earth_Work.xlsx"

on_going_path = ''

In [20]:
job_codes = job_code(completed_path)
newest = lat_old(job_codes)
# newest
full_df = files_to_dfs(newest)
# oldest_df = files_to_dfs(oldest)

In [21]:
full_df['Ph Int'] = full_df['Phase Code'].astype(str).str[0]
full_df = full_df[~full_df['updated'].isnull()]
full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
full_df['month'] = pd.DatetimeIndex(full_df['updated']).month
# full_df['Est unit $'] = full_df['Total Budget'] / full_df['Qty']
# full_df['Act unit $'] = full_df['Actual Cost$'] / full_df['HJ Qty Comp']
full_df = full_df.sort_values(by='updated', ascending=False)

no_dup = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='last')
no_dup['Qty'] = no_dup['Qty'].astype(float)
no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
no_dup['Total Budget'] = no_dup['Total Budget'].astype(float)

most_recent = no_dup.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]

earth_work = most_recent[most_recent['Ph Int'] == '3']

earth_work['crew'] = 'misc.'

for key, values in crew_dict.items():
    earth_work.loc[earth_work['Description'].str.lower().str.contains('|'.join([x.lower() for x in values])), "crew"] = key

reduced = earth_work.loc[:, earth_work.columns.notna()]
reduced = reduced.drop(['% Comp', 'Notes','Lab','Eq', 'Reclass'], axis=1)

C:\Users\willd\AppData\Local\Temp\ipykernel_11412\4185029306.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Qty'] = no_dup['Qty'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_11412\4185029306.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_11412\4185029306.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [26]:
test = reduced
test.loc[test['Total Budget'] > 0, 'Est UC'] = round(test['Total Budget'] / test['Qty'], 2)
test.loc[test['Actual Cost$'] > 0, 'Act. UC'] = round(test['Actual Cost$'] / test['HJ Qty Comp'], 2)

In [33]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
test.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [8]:
file = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
xl = pd.read_excel(file ,sheet_name=1)
df = pd.DataFrame(xl)

In [11]:
# df = df[df[['Est UC', 'Act. UC']].astype(float)
df[['Est UC', 'Act. UC']] = df[['Est UC', 'Act. UC']].apply(pd.to_numeric, errors='coerce')

In [16]:
file = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
xl1 = pd.read_excel(file ,sheet_name=0)
most_recent = pd.DataFrame(xl)

In [17]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
df.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [80]:
file = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections\Projection 10304-50 Greyhound-Crossland 20220126.xlsx"
xl = pd.read_excel(file ,sheet_name=1)
most_recent = pd.DataFrame(xl)

ValueError: Worksheet index 1 is invalid, 1 worksheets found

In [79]:
most_recent

,Projection 10304- 50 - Mile High Greyhound Park Development-Crossland,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,"1/26/22 Erica S, Erica H, Hutto, Steve, Tracy,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Current Projection 1/26/22,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,Budget Remaining,Over/ (Under) Budget,% Comp,NaN,Over/ (Under) Variance
3,1001,Contingency,NaN,1,NaN,LS,0,NaN,38768.82,NaN,...,0,NaN,10669.01,NaN,10669.01,0,-28099.81,1,NaN,0
4,1005,Travel/Per Diem/Field Office,NaN,1,NaN,LS,0,NaN,6375,NaN,...,0,NaN,1135.99,NaN,1135.99,0,-5239.01,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,Totals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3451170.61,-4484.29,...,20542.86,NaN,3028415.27,NaN,3036668.18,8252.91,-410018.14,0.997282,NaN,10253.99
267,Source Data Totals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3451170.61,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10253.99
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ew_ = [x for x in job_code(completed_path) if '-30' in x]
unique_cod = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_)]))
ew_code = [x for x in test['job'] if '-30' in x]
unique_codes = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_code)]))
print(unique_codes)
print(unique_cod)

In [74]:
ful = files_to_dfs(newest)
ew_code = [x for x in ful['job'] if '-30' in x]
unique_codes = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_code)]))

In [75]:
print(len(unique_codes))
print(len(unique_cod))

23
26


In [8]:
# lat_test = latest_df
# old_test = oldest_df

In [31]:
# rename columns for oldest/newest

# def old_new_col(df, chron):
#     df = df.rename(columns={"Qty": (chron + " Qty"), "Total Budget": (chron + " Total Budget")})
#     return df 
    

In [53]:
# lat = old_new_col(lat_test, "Latest")
# old = old_new_col(old_test, "Oldest")
# old = old[['Phase Code', 'Description', 'Oldest Qty', 'Oldest Total Budget', 'job']]

In [54]:
# full_df = lat.join(old.set_index('Phase Code'), on=['Phase Code', 'Description'], validate='m:1')
# full_df = pd.merge(lat,old, how='left', on=['Phase Code', 'Description'])
# full_df = pd.merge(lat,old, how='left', 
#                   left_on=['Phase Code', 'Description', 'job'],
#                   right_on=['Phase Code', 'Description', 'job'])

In [4]:
# files = []
# for item in os.listdir(completed_path):
#     job_code = item.split(' ')[1]
#     files.append(job_code)
# files= pd.unique(files)
    
#     full_path = os.path.join(completed_path, item)

In [5]:
# latest_file_list = []
# for file in files:
#     job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
#     latest_file = max(job_files, key=os.path.getctime)
#     latest_file_list.append(latest_file)
# latest_file_list = pd.unique(latest_file_list)   

In [6]:
# df_list = []
# for item in latest_file_list:
#     num_sheet = len(pd.ExcelFile(item).sheet_names)
#     for sheet in range(0,num_sheet):
#         xl = pd.read_excel(item ,sheet_name=sheet)
#         df = pd.DataFrame(xl)
#         job, date = job_date(df)
#         df.columns = df.iloc[2]
#         df = df.drop(df.index[:3])
#         df = df.iloc[:, :27]
#         df['updated'] = date
#         df['job'] = job
#         df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
#         df = df.dropna(subset=['Qty'])
#         df = df.reset_index(drop=True)
#         df_list.append(df)

In [56]:
# full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
full_df['Ph Int'] = full_df['Phase Code'].astype(str).str[0]
full_df = full_df[~full_df['updated'].isnull()]
full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
full_df['month'] = pd.DatetimeIndex(full_df['updated']).month
# full_df['Est unit $'] = full_df['Total Budget'] / full_df['Qty']
# full_df['Act unit $'] = full_df['Actual Cost$'] / full_df['HJ Qty Comp']
full_df = full_df.sort_values(by='updated', ascending=False)

C:\Users\willd\AppData\Local\Temp\ipykernel_6548\2810187377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
C:\Users\willd\AppData\Local\Temp\ipykernel_6548\2810187377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['month'] = pd.DatetimeIndex(full_df['updated']).month


In [57]:
# full_df = full_df[~full_df['updated'].isnull()]
# first = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='first')
# last = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='last')

######

#####



#######
no_dup = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='last')
no_dup['Latest Qty'] = no_dup['Latest Qty'].astype(float)
no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
no_dup['Latest Total Budget'] = no_dup['Latest Total Budget'].astype(float)

C:\Users\willd\AppData\Local\Temp\ipykernel_6548\3602641939.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Latest Qty'] = no_dup['Latest Qty'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_6548\3602641939.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_6548\3602641939.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [59]:
most_recent = no_dup.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Latest Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]
# df.drop_duplicates(subset=['IDName'], keep='last')


In [60]:
# subset of just earthwork
earth_work = most_recent[most_recent['Ph Int'] == '3']

In [61]:
crew_dict = {"Scarify":["Scarify"], "Strip Site":["Strip Site"], 
             "Soil Correction":["Soil Correction", "OX","OE", "Over Ex", "Excavate & Re-compact", "Over-Ex", "Excavate & Recompact"],
             "Ex Retainining Wall":["Excavate Site Retaining", "Excavate MSE", "Exc Retaining", "Exc Site Retn Walls"],
             "Export":["Export", "Exp"], "Import":["Import", "Imp"],
             "Ex Frost footing": ["Exc Fro", "Excavate Fro", "Footing Excavation"], 
             "Ex Trash Enc":["Excavate Trash", "Exc Trash"],
             "Cut to Fill":["Cut To Fill", "Utility Spoils - Site Fill", "Building Pad Fill w/ Spoils"],
             "Cut to Stockpile":["Cut to Stockpile", "Load & Stockpile"],
             "Handle spoils":["Handle Utility Spoils", "Handle Foundation Spoils", "Handle Spoils", "Handle & Stockpile"],
             "Double Handle":["Double Handle"], "Cut in C&G":["Cut In C", "Cut-in C", "Grade C&G"],
             "Ex Elevator":["Excavate Elevator"],
             "Grade Landscape":["grade Landscape", "grade Green", "Landscape"],
             "Grade Parkinglots and Roads":["Grade Boulevards","Grade Asphalt", "Grade Parking", "Grade Pkg", "Grade Main Roads"],
             "Grade SW":["Grade Sidewalks", "Grade Walks", "Exterior Slabs", "grade Brick Pavers", "Grade SW"],
             "Export":["Export"],
             "Grade BLDG": ["Grade Bldg", "Grade Building", "Grade Structural"],
             "Drain Tile":["Drain"],
             "Backfill Int":["Backfill Int", "Footing Backfill"],
             "Backfill Ext":["Backfill Ext"],
             "Backfill Elevator":["Backfill Elevator"],
             "Backfill Trash":["Backfill Trash"], "Backfill Frost": ["Backfill Frost"],
             "High Backfill": ["High Backfill"], "Grade Class X":["Grade Class"],
             "Excavate Interior":["Excavate Int", "Excavate Thickened", "Excavate Grade Beams", "Exc Int"],
             "Excavate Exterior":["Excavate Ext", "Exc Ext"], "Grade Pond":["Basin Grading","Grade Pond" , "Grade Detention Pond"],
             "Assist shoring":["Assist Shoring"],
             "Stairs":["Stairs"], "Place Strippings":["Place Stripping"],
             "Street Sweep":["Street Sweeping"]}

In [62]:
test = earth_work
test['crew'] = 'misc.'

C:\Users\willd\AppData\Local\Temp\ipykernel_6548\2156627274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crew'] = 'misc.'


In [63]:
for key, values in crew_dict.items():
    test.loc[test['Description'].str.lower().str.contains('|'.join([x.lower() for x in values])), "crew"] = key

In [64]:
reduced = test.loc[:, test.columns.notna()]
reduced = reduced.drop(['Notes','Lab','Eq', 'Reclass'], axis=1)

In [60]:
# v = most_recent.Description.value_counts()
# agg_df = most_recent[most_recent.Description.isin(v.index[v.gt(3)])]

In [65]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
reduced.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [66]:
# agg_df.info()